# Работа №1 - Поиск ассоциативных правил

_Вариант 5. Сформировать набор из 30 транзакций на основе продуктов, употребляемых на завтрак, обед и ужин._

In [1]:
! echo '[["Мясо", "Соус", "Вино"],["Мясо", "Соус", "Картофель", "Масло"],["Картофель", "Масло"],["Картофель", "Масло"],["Вино", "Яйца", "Бекон"],["Овощи", "Мясо", "Специи"],["Яйца", "Бекон", "Овощи", "Мясо", "Специи"],["Яйца", "Бекон", "Овощи", "Мясо", "Специи"],["Лапша", "Кетчуп"],["Пельмени", "Майонез", "Кетчуп"],["Пельмени", "Майонез", "Кетчуп"],["Овощи", "Пельмени", "Майонез", "Кетчуп"],["Овощи", "Соус", "Вино"],["Кетчуп"],["Кетчуп", "Лапша", "Пельмени", "Майонез"],["Кетчуп", "Лапша", "Пельмени", "Майонез"],["Кетчуп", "Лапша", "Мясо", "Соус", "Вино"],["Овощи", "Мясо", "Соус", "Вино"],["Майонез"],["Вино"],["Овощи", "Масло"],["Овощи", "Масло", "Картофель"],["Овощи", "Масло", "Картофель", "Мясо", "Соус"],["Овощи", "Бекон", "Яйца"],["Бекон", "Яйца"],["Яйца", "Кетчуп", "Лапша"],["Мясо"],["Мясо", "Масло"],["Мясо", "Масло", "Картофель"],["Масло", "Картофель"]]' > data/data_set.json
! curl https://raw.githubusercontent.com/viktree/curly-octo-chainsaw/master/BreadBasket_DMS.csv > ./data/BreadBasket_DMS.csv
! pip install apriori_python efficient-apriori fpgrowth-py pyarmviz pandas

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  693k  100  693k    0     0   773k      0 --:--:-- --:--:-- --:--:--  772k


In [14]:
from lib import datasets
from pandas import pandas
from fpgrowth_py import fpgrowth
from efficient_apriori import apriori

In [15]:
repo_ds = datasets.from_csv("./data/BreadBasket_DMS.csv")
test_ds = datasets.from_json("./data/data_set.json")

In [21]:
_, test_rules = apriori(test_ds, min_support=0.15, min_confidence=0.6)
_, repo_rules = apriori(repo_ds, min_support=0.01, min_confidence=0.6)
repo_rules

[{Toast} -> {Coffee}]

In [19]:
_, test_rules_fpg = fpgrowth(test_ds, minSupRatio=0.15, minConf=0.6)
_, repo_rules_fpg = fpgrowth(repo_ds, minSupRatio=0.01, minConf=0.6)
repo_rules_fpg

[[{'Toast'}, {'Coffee'}, 0.7044025157232704]]